# Access Google Drive in TIR Notebooks
This document provides a step-by-step guide on accessing your Google Drive in TIR Notebooks. First we'll setup google OAuth2 credentials for authorization and then use Google Drive APIs to list files, search for specific files, upload and download files, and create folders in Google Drive.

## Pre-requisites
1. **Google Cloud Console Account**: Make sure you have a Google account and are able to access the [Google Cloud Console](https://console.cloud.google.com).

2. **Google Drive API Enabled**: Ensure the Google Drive API is enabled for your project in the Google Cloud Console. If not, enable from [here](https://console.cloud.google.com/apis/api/drive.googleapis.com/).


## Setup Instructions
### Step 1: Create OAuth2 Credentials
To interact with Google Drive APIs securely, you'll need to create OAuth2 credentials from the Google Cloud Console. These credentials provide authorization to access files from your Google Drive.

Follow these steps to set up OAuth2 credentials:

1. Go to the [Google Cloud Console](https://console.cloud.google.com).

2. Navigate to APIs & Services > Credentials > Create credentials > OAuth 2.0 Client IDs.

3. Choose Web application as the application type.

4. Set the Authorized JavaScript origins to `https://notebooks.e2enetworks.com:443`.

5. Set the Authorized redirect URI to `https://gpu-notebooks.e2enetworks.com/oauth2callback/`.

6. Download the JSON file containing your OAuth2 credentials.

7. Upload the downloaded credentials file to this JupyterLab Notebook using the top arrow icon in the toolbar.

Input the path to the uploaded credentials file in the code cell below.


In [ ]:
# enter your credentials files path here
creds_file_path = "path/to/your/credentials_file.json"

### Step 2: Set Up OAuth Consent Screen
1. Navigate to APIs & Services > OAuth consent screen.

2. Set the User Type to External and ensure that the Publishing status is set to In Production or you will have to re-download the token every 7 days.

### Step 3: Install the required Python Libraries
To interact with the Google Drive API, you need to install several Python libraries. Run the below command

In [ ]:
!pip3 install google-auth google-api-python-client google-auth-oauthlib

### Step 4: Configure and Authenticate the Google Drive API Client
Before accessing files from Google Drive, you need to set up and authenticate the Google Drive API client in this Notebook.

Execute the following code to set up the Google Drive API client.
- Follow the authorization URL displayed in the output and select your Google Account.
- Upon successful authorization, you will get a code. Copy the authorization code and enter it in the box that requests for the authorization code to complete the authentication process.

In [ ]:
import io
import os
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

# Load existing credentials if available
creds = None
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)

# If credentials are missing or expired, initiate OAuth flow
if not creds or not creds.valid:
    flow = InstalledAppFlow.from_client_secrets_file(
        creds_file_path,
        SCOPES,
        redirect_uri='https://gpu-notebooks.e2enetworks.com/oauth2callback/'
    )
    authorization_url, _ = flow.authorization_url(prompt='consent')
    print(f'Please go to this URL: {authorization_url}')
    
    authorization_code = input('Enter the authorization code: ')
    flow.fetch_token(code=authorization_code)
    creds = flow.credentials
    
    # Save the updated credentials for future use
    with open("token.json", "w") as token:
        token.write(creds.to_json())

# Build the Drive API service
drive_service = build('drive', 'v3', credentials=creds)


## Performing Actions with Google Drive API
### Searching for a Specific File/Folder
To search for a file/folder by name, use the following query:
> **Note:** To list all files and folders, use an empty query (`""`) when searching for files.

In [ ]:
# Define your query
query = "name = 'example.txt'" # Change 'example.txt' to your file name or empty string("") to list all files

results = (drive_service.files().list(q=query, fields="files(id, name, mimeType)").execute())
items = results.get("files", [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(f"{item['name']} ({item['id']}) - {item['mimeType']}")

### List Files in specific Folder
To list all files within a specific folder, first obtain the folder ID by executing the code above. Then, use that folder ID in the code below to list all files in that folder.

In [ ]:
# Define the folder ID where you want to list files
folder_id = 'your_folder_id_here'

# Define your query with the folder ID
query = f"'{folder_id}' in parents"

results = drive_service.files().list(q=query, fields="files(id, name, mimeType)").execute()
items = results.get("files", [])

if not items:
    print('No files found in the folder.')
else:
    print('Files in the folder:')
    for item in items:
        print(f"{item['name']} ({item['id']}) - {item['mimeType']}")

### Downloading Files from Google Drive
To download a file, use the following code:

In [ ]:
file_id = 'file_id_to_download'
file_path = 'path/to/save/downloaded/file.txt'

request = drive_service.files().get_media(fileId=file_id)
fh = io.FileIO(file_path, mode='wb')
downloader = MediaIoBaseDownload(fh, request)
done = False

while not done:
    status, done = downloader.next_chunk()

### Uploading Files to Google Drive
To upload a file to a specific folder in Google Drive, use the following code:

In [ ]:
file_metadata = {
    'name': 'MyFile.txt',    
    'parents': '<folder_id>'  # ID of the folder where you want to upload
}
file_path = 'path/to/your/local/file.txt'

media = MediaFileUpload(file_path, mimetype='text/plain')

file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

### Creating a Folder in Google Drive
To create a new folder, use the following code:

In [ ]:
folder_metadata = {
    'name': 'MyFolder',  
    'parents': '<parent_folder_id>'  # ID of the parent folder (optional)
}

folder = drive_service.files().create(body=folder_metadata, fields='id').execute()

## Troubleshooting and Best Practices
1. **OAuth2 Flow Issues**: Ensure the redirect URI is correctly set up in the Google Cloud Console.

2. **Token Expiration**: If your publishing status is not set to 'In Production', your token will expire in 7 days, requiring you to re-authenticate and recreate the credentials.
## References
* [Google Drive API Documentation](https://developers.google.com/drive/api/guides/about-sdk)

* [Google Cloud Console](https://cloud.google.com/docs)

* [Python Google API Client Library](https://googleapis.github.io/google-api-python-client/docs/)
